In [1]:
import pandas as pd
import numpy as np
import orquest_csv_utils as ocu

utils imported


In [2]:
# Load relevant data
measures = pd.read_csv('measures_rich.csv', dtype=ocu.dftypes['measures'])
current_name = 'measures'
for col in measures.columns:
   if col in ocu.tcasts[current_name].keys():
       match ocu.tcasts[current_name][col]:
           case 'decimal_with_comma':
               ocu.cast_comma_decimal_as_float(measures,col)
           case 'dmy_hm_datetime':
               ocu.cast_d_m_y_h_m_as_datetime(measures,col)
           case 'ymd_datetime':
               ocu.cast_y_m_d_as_datetime(measures,col)
measures.dtypes

measure             object
date        datetime64[ns]
value              float64
store_id             int32
dtype: object

In [3]:
# Split datetime into date and time
measures['time'] = measures['date'].dt.time
measures['date'] = measures['date'].dt.date
measures

,measure,date,value,store_id,time
0,FOOTFALL,2023-11-02,55.0,1,10:00:00
1,FOOTFALL,2023-11-02,55.0,1,10:30:00
2,FOOTFALL,2023-11-02,67.0,1,11:00:00
3,FOOTFALL,2023-11-02,67.0,1,11:30:00
4,FOOTFALL,2023-11-02,56.0,1,12:00:00
...,...,...,...,...,...
11079,TICKETS,2024-01-31,19.0,2,20:00:00
11080,TICKETS,2024-01-31,19.0,2,20:30:00
11081,TICKETS,2024-01-31,4.0,2,21:00:00
11082,TICKETS,2024-01-31,4.0,2,21:30:00


In [4]:
measures_split = measures.pivot_table(index=['date','time','store_id'],columns='measure',values='value',aggfunc='sum')

In [5]:
measures_split.sort_values(by=['date','time'],inplace=True)

measures_split['sales_cumulative'] = measures_split.groupby('date')['SALES'].cumsum()

measures_split['tickets_cumulative'] = measures_split.groupby('date')['TICKETS'].cumsum()

measures_split['footfall_cumulative'] = measures_split.groupby('date')['FOOTFALL'].cumsum()


In [6]:
measures_split

measure                       FOOTFALL     SALES  TICKETS  sales_cumulative  \
date       time     store_id                                                  
2023-11-01 10:00:00 2             33.0   277.860      8.0           277.860   
           10:30:00 2             33.0   277.860      8.0           555.720   
           11:00:00 2             89.0   743.090     24.0          1298.810   
           11:30:00 2             89.0   743.090     24.0          2041.900   
           12:00:00 2            161.0  1000.845     42.0          3042.745   
...                                ...       ...      ...               ...   
2024-01-31 20:00:00 2             60.0   233.615     19.0         15024.525   
           20:30:00 2             60.0   233.615     19.0         15258.140   
           21:00:00 2             16.0    62.370      4.0         15320.510   
           21:30:00 2             16.0    62.370      4.0         15382.880   
           22:00:00 2             16.0    62.370      4.0         15445.250   

measure                       tickets_cumulative  footfall_cumulative  
date       time     store_id                                           
2023-11-01 10:00:00 2                        8.0                 33.0  
           10:30:00 2                       16.0                 66.0  
           11:00:00 2                       40.0                155.0  
           11:30:00 2                       64.0                244.0  
           12:00:00 2                      106.0                405.0  
...                                          ...                  ...  
2024-01-31 20:00:00 2                      755.0               2140.0  
           20:30:00 2                      774.0               2200.0  
           21:00:00 2                      778.0               2216.0  
           21:30:00 2                      782.0               2232.0  
           22:00:00 2                      786.0               2248.0  

[3834 rows x 6 columns]

In [7]:
measures_split.to_csv('store_measures.csv')